In [1]:
import sys
sys.path.append('../../30_data_tools/')
from helper import load_dotenv

In [2]:
from pathlib import Path
import torch
import plotly.express as px
from PIL import Image
from tqdm.auto import tqdm

In [3]:
# import YOLO model
from ultralytics import YOLO, settings
import mlflow

In [4]:
import os

In [5]:
dotenv = load_dotenv()

In [6]:
mlflow.set_tracking_uri(dotenv['MFLOW_DB_PATH'])
mlflow.set_experiment("moire_detection")

<Experiment: artifact_location='/Users/frederic.birwe/Documents/GitHub/master-thesis_moire_detection/60_gebastel/Model/mlruns/1', creation_time=1709374525946, experiment_id='1', last_update_time=1709374525946, lifecycle_stage='active', name='moire_detection', tags={}>

In [7]:
os.environ.MLFLOW_EXPERIMENT_NAME = 'moire_detection'
os.environ.MLFLOW_RUN = '24-03-02_test'

In [8]:
settings.update({'mlflow': True})

In [ ]:
# input size 640x640

In [10]:
results

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x2c4209ba0>
fitness: 0.845818817615509
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6916376352310181, 'metrics/accuracy_top5': 1.0, 'fitness': 0.845818817615509}
save_dir: PosixPath('/Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/train24')
speed: {'preprocess': 0.0002936619084055831, 'inference': 55.232151044785766, 'loss': 3.239834350160605e-05, 'postprocess': 4.9843605387086236e-05}
top1: 0.6916376352310181
top5: 1.0

In [16]:
with mlflow.start_run():
    # Load and train your YOLOv8 model
    model = YOLO('yolov8m-cls.pt')
    results = model.train(data='./dataset_test/', epochs=2)

    # Log parameters, metrics, and model
    mlflow.log_params({"epochs": 3, "model": "yolov8m-cls.pt"})
    mlflow.log_metrics({"top_1": results.top1})
    mlflow.pytorch.log_model(model, "model")

    # End the run
    mlflow.end_run()

New https://pypi.org/project/ultralytics/8.1.20 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.184 🚀 Python-3.10.11 torch-2.0.1 CPU (Apple M1 Pro)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=./dataset_test/, epochs=2, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, clas

TypeError: cannot pickle 'torch._C.Generator' object

In [ ]:
metrics = model.val() # no arguments needed, dataset and settings remembered
metrics.top1 # top1 accuracy
metrics.top5 # top5 accuracy

In [ ]:
str(metrics.save_dir)

In [ ]:
!open '/Users/frederic.birwe/Desktop/test/pagina-finder-webapp/runs/classify/val19'

In [ ]:
metrics.confusion_matrix.matrix

In [ ]:
model = YOLO(dotenv['MODEL_DIR'] / 'yolov8_moires_24-02-22.pt')

In [ ]:
moire_img = list( Path('./dataset/test/moire/').glob('./*.jpg') )
non_moire_img = list( Path('./dataset/test/no_moire/').glob('./*.jpg') )

data = moire_img + non_moire_img
labels = [1 for i in moire_img] + [0 for i in non_moire_img]

In [ ]:
#model.val()
predictions = []
target_labels = []

for entry in tqdm(data):
    pred = model.predict(entry, verbose=False)[0].probs.data
    predictions.append(pred)
    target_labels.append( 1 - int(torch.argmax(pred)) )

In [ ]:
str(metrics.save_dir)

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(labels)):
    # Positive == 1, Negative == 0
    res = labels[i] - target_labels[i]
    if res == 0:
        if labels[i] == 1:
            TP += 1
        else:
            TN += 1
    elif res == -1:
        FP += 1
    else:
        FN += 1

recall = TP / (TP + FN)
precision = TP / (TP + FP)
correct_values = TP + TN
accuracy = correct_values / len(labels)

In [ ]:
TP, TN, FP, FN

In [ ]:
TP / (TP + FN)

In [ ]:
recall, precision

In [ ]:
f"{(TP + TN)}/{len(labels)}, {accuracy}"

In [ ]:
import shutil

In [ ]:
for i in tqdm(range(1,len(labels))):
    # Positive == 1, Negative == 0
    res = labels[i] - target_labels[i]
    if res == 1:
        dir_path = Path('./temp/FN/')

        if dir_path.exists() == False:
            dir_path.mkdir()
    elif res == -1:
        dir_path = Path('./temp/FP/')

        if dir_path.exists() == False:
            dir_path.mkdir()

    if res != 0:
        out_path = dir_path / data[i].name
        shutil.copy( data[i], out_path )

In [ ]:
from random import randrange

In [ ]:
model.predict(data[i], verbose=False)[0].probs.data

In [ ]:
model.predict(Path('./dataset/train/moire/508890.HBZP_1_M_270.p1.soft_light.1.4c_600.jpg.0908.jpg'), verbose=False)[0].probs.data

In [ ]:
classes = {
    '1' : [],
    '0.5' : [],
    '-0.5' : [],
    '-1' : []
}

for i in range(len(predictions)):
    diff = float(predictions[i][0] - predictions[i][1])

    if diff > 0.5:
        classes['1'].append(i)
    elif diff >= 0:
        classes['0.5'].append(i)
    elif diff >= -.5:
        classes['-0.5'].append(i)
    else:
        classes['-1'].append(i)

In [ ]:
for c in classes:
    acc = len([True for i in classes[c] if labels[i] - target_labels[i] == 0]) / len(classes[c])

    print(c, acc)

In [ ]:
i = randrange(len(data))
print(i, data[i].name)
print(predictions[i], labels[i],target_labels[i])
Image.open(data[i])

In [ ]:
import pandas as pd

In [ ]:
res_df = pd.DataFrame(
    [(float(predictions[i][0]),float(predictions[i][1]),labels[i],target_labels[i]) for i in range(len(labels))],
    columns=['pred_moire','pred_no_moire','label','target_label']
)

res_df.loc[
    :,
    'pred_diff'
] = res_df.pred_moire - res_df.pred_no_moire
res_df.loc[
    :,
    'pred_correct'
] = res_df.label == res_df.target_label
res_df.loc[
    :,
    'pred_diff_abs'
] = res_df.pred_diff.abs()

In [ ]:
res_df.corr()

In [ ]:
px.scatter(
    y=res_df.pred_diff_abs,
    color=res_df.pred_correct
)

In [ ]:
px.scatter(
    sorted([p[0]-p[1] for p in predictions], reverse=True)
)

In [ ]:
px.scatter(
    [p[0] for p in predictions]
)

In [ ]:
px.scatter(
    [p[1] for p in predictions]
)